In [110]:
# Import relevant packages
import sklearn
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

In [159]:
# read the data set

dataset = []
fns = []
for line in open('dataset-example.csv'):
    fn, txt = line.split('|')
    fns.append(fn)
    dataset.append(txt)

In [160]:
# Perform tfidf vecotrization

count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(dataset)
tf = TfidfVectorizer()
tfidf_matrix = tf.fit_transform(dataset)

In [161]:
# Feature score for a document
def tfidfscores(doc):
    feature_names = np.array(tf.get_feature_names())
    feature_index = tfidf_matrix[doc,:].nonzero()[1]
    tfidf_scores = list(zip(feature_names[feature_index], [tfidf_matrix[doc, x] for x in feature_index]))
    return sorted(tfidf_scores, reverse=True, key=lambda x:x[1])

In [166]:
# sum of feature scores for multiple documents
def tfidfsum(docs, tf, tfidf_matrix):
    feature_names = np.array(tf.get_feature_names())
    feature_sum = np.zeros(tfidf_matrix[0,:].shape)
    for doc in docs:
        feature_sum += tfidf_matrix[doc,:]
    feature_index = feature_sum.nonzero()[1]
    tfidf_scores = list(zip(feature_names[feature_index], [float(feature_sum[:,x]) for x in feature_index]))
    return sorted(tfidf_scores, reverse=True, key=lambda x:x[1])

In [167]:
tfidfsum((45,), tf, tfidf_matrix)

[('100z', 0.47297681863515967),
 ('xbf', 0.2719316280319375),
 ('navigationlandingpage', 0.26329924349992306),
 ('file', 0.25242879037344956),
 ('06', 0.24122782363508824),
 ('01', 0.2320791451017774),
 ('957z', 0.18286674095289332),
 ('278z', 0.17831090991578907),
 ('data', 0.17405935882633305),
 ('raw', 0.17405935882633305),
 ('2019', 0.1547194300678516),
 ('filename', 0.14580217006568727),
 ('ntfs', 0.14580217006568727),
 ('timestamps', 0.14580217006568727),
 ('14', 0.14492840519793063),
 ('177389', 0.13164962174996153),
 ('2200', 0.13164962174996153),
 ('311080', 0.13164962174996153),
 ('49f39a7481a7e77140421355dc8398ce3ab8bfa45f1cbaf422041a1240e34cbf',
  0.13164962174996153),
 ('7e66eb3ea43016930b37c2f5562c4af5', 0.13164962174996153),
 ('f36ae68a8d7d31753f462d68f3d4d49e65c5c9fe', 0.13164962174996153),
 ('01t05', 0.12884424146219328),
 ('14t16', 0.10888156529043638),
 ('secureassessmentbrowser_cw5n1h2txyewy', 0.10329076730357614),
 ('59', 0.09100572474976847),
 ('00', 0.08982456862